In [186]:
import os
import time
import openai
import tqdm
path = "/Users/aditiraghavan/Downloads/RTLLM/_chatgpt4"

Designs that I want to synthesize

In [338]:
design_name = ['accu', 'adder_8bit', 'adder_16bit', 'adder_32bit', 'adder_pipe_64bit', 'asyn_fifo', 'calendar', 'counter_12', 'edge_detect',
                'freq_div', 'fsm', 'JC_counter', 'multi_16bit', 'multi_booth_8bit', 'multi_pipe_4bit', 'multi_pipe_8bit', 'parallel2serial' , 'pe' , 'pulse_detect', 
                'radix2_div', 'RAM', 'right_shifter',  'serial2parallel', 'signal_generator','synchronizer', 'alu', 'div_16bit', 'traffic_light', 'width_8to16']

Find all design descriptions, add it to a dictionary.

In [360]:
design_descriptions = dict()
for design in design_name:
    if not (os.path.exists(f"{design}/design_description.txt")):
        print(f"Description for {design} not found")
        continue
    with open(f"{design}/design_description.txt", "r") as f:
        design_descriptions[design] = f.read()

print(design_descriptions['adder_8bit'])

Please act as a professional verilog designer.

Implement a module of an 8-bit adder with multiple bit-level adders in combinational logic. 

Module name:  
    adder_8bit               
Input ports:
    a[7:0]: 8-bit input operand A.
    b[7:0]: 8-bit input operand B.
    cin: Carry-in input.
Output ports:
    sum[7:0]: 8-bit output representing the sum of A and B.
    cout: Carry-out output.

Implementation:
The module utilizes a series of bit-level adders (full adders) to perform the addition operation.

Give me the complete code.
 


Add path for where results should be stored

In [350]:
path_for_verilog = os.path.join(path, f"t21")
print(os.path.exists(path_for_verilog))
print(path_for_verilog)


True
/Users/aditiraghavan/Downloads/RTLLM/_chatgpt4/t21


In [359]:
openai.api_key = None

Initalize dicts for the output from chatGPT

In [351]:
code_gen = dict()
prompt_gen = dict()

Below is the verilog example and design description I used when doing one-shot. The design was verified using the benchmark's testbench

In [260]:
verilog_code = """module pulse_detect(
    input  logic clk,
    input  logic rst_n,
    input  logic data_in,
    output logic data_out
);

    // State encoding
    enum logic [1:0] {IDLE, S0, S01} state, next_state;

    // State register
    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            state <= IDLE;
        end else begin
            state <= next_state;
        end
    end

    //State transition logic
    always_comb begin
        case (state)
            IDLE: begin
                if (data_in == 1'b0)
                    next_state = S0;
                else
                    next_state = IDLE;
            end
            S0: begin
                if (data_in == 1'b1)
                    next_state = S01;
                else
                    next_state = S0; 
            end
            S01: begin
                if (data_in == 1'b0)
                    next_state = S0;  // Move to S0 to handle overlapping patterns
                else
                    next_state = IDLE;
            end
            default: next_state = IDLE;
        endcase
    end

    //Output logic
    assign data_out = (state == S01 && data_in == 1'b0);
endmodule
"""
verilog_design_description = """Please act as a professional verilog designer.

Implement a module for pulse detection. The 1-bit input signal data_in is a continuous input, which is triggered by clk. data_out is 1 when 0->1->0 is seen on data_in. Data_out is 1 on the same cycle 010 is seen. This requires 3 cycles. The pattern can overlap, for example, if data_in is 010101, the data_out is 00101. If data_in is 0110, data_out is 0000.


Module name:  
    pulse_detect

Input ports：
    clk: Clock signal.
    rst_n: Reset signal (active low).
    data_in: One-bit input signal.

Output ports：
    data_out: Output signal indicating the presence of pulses.

Implementation:
Declare the module pulse_detect with input and output ports as specified in the ports statement.
Declare a register state to keep track of the current state of the pulse detection process.
Inside an always block, sensitive to the positive edge of the clk signal and the negative edge of the rst_n signal, implement the logic for pulse detection and output generation.
In the reset condition (~rst_n), set the state register to the initial state and set the data_out output to 0, indicating no pulse. Continue the detection process for the remaining states. For each state, check the value of data_in and transition to the next state accordingly. If the current state satisfies the last state of a "pulse", set data_out to 1, indicating the end of a pulse. Otherwise, set data_out to 0.

Give me the complete code."""

augment_text_generate adds my one shot example and additional instruction prompting to the original design description. Commented out lines show other techniques which were less successful. This includes chain of thought, and other instruction prompts

In [332]:
def augment_text_generate(prompt):
    out_prompt = verilog_design_description 
    out_prompt += "\n"
    out_prompt += verilog_code
    out_prompt += "\n"

    out_prompt += prompt
    out_prompt += "Ensure that you do not have syntax errors in the code."
    out_prompt += "All output logic should be blocking."

    #Commented out code as not used in final prompting strategy, but were tried out

    # out_prompt = "Use the exact module names and signal names as given in the description.\n"
    # out_prompt += "Let's think step by step before writing the code. \n"
    # out_prompt += "Identify edge cases and complex scenarios that you need to consider.\n"
    # out_prompt += "Give me the complete code."
    
    return out_prompt 


In [352]:

for i in tqdm.tqdm(range(0, len(design_name))):
    design = design_name[i]
    send_gpt = design_descriptions[design]
    send_gpt = augment_text_generate(send_gpt)
    completion = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful chatbot"},
                {"role": "user", "content": send_gpt},
            ],
        )
    code_gen[design] = completion.choices[0].message.content
    

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [14:05<00:00, 29.15s/it]


Below is an example of the output given by GPT-4

In [353]:
print(code_gen['adder_8bit'])


Certainly! Let's start with implementing the `adder_8bit` as requested using full adders for bit-by-bit computation of the sum between two 8-bit input numbers. This implementation will be in Verilog and showcase handling of carry propagation through the 8 bits.

```verilog
module adder_8bit(
    input  wire [7:0] a,    // 8-bit input operand A
    input  wire [7:0] b,    // 8-bit input operand B
    input  wire       cin,  // Carry-in input
    output wire [7:0] sum,  // 8-bit sum of A and B
    output wire       cout  // Carry-out
);

    // Internal variables for carry
    wire [8:0] carry;   // wire array to store carry out from each bit addition, carry[8] is cout

    // Assign the first carry input
    assign carry[0] = cin;

    // Generate the full adders
    genvar i;
    generate
        for (i = 0; i < 8; i = i + 1) begin: full_adder_gen
            full_adder fa(
                .a(a[i]),
                .b(b[i]),
                .cin(carry[i]),
                .sum(sum[i]),

Write the raw file somewhere just for reference if needed

In [354]:
# Write code gen to a file, with file id in file name
with open(f"code_gen_{time.time()}.txt", "w") as f:
    #Put send gpt
    f.write(send_gpt)
    f.write("\n\n")
    for design in code_gen:
        f.write(f"{design}\n")
        f.write(f"{code_gen[design]}\n")
        f.write("\n\n")
    
code_gen_parse = dict()

Parse, output to remove all additional comments and just capture the verilog code. There may be multiple modules in one reponse so post-processing needs to support that

In [357]:
import re
def extract_verilog_code(text):
    # Define the regular expression pattern to match code blocks
    pattern = r"```verilog\n(.*?)\n```"
    
    # Use re.findall to extract all occurrences of the pattern
    verilog_codes = re.findall(pattern, text, re.DOTALL)
    
    # Return the list of extracted Verilog code blocks
    return verilog_codes

for design in design_name:
    code_gen_parse[design] = extract_verilog_code(code_gen[design])


Write every design into the corresponding file name

In [358]:
#Write the verilog code to the file
for design in design_name:
    with open(f"{path_for_verilog}/{design}.v", "w") as f:
        for code in code_gen_parse[design]:
            f.write(code)
            f.write("\n")

The code below was used for trying self planning as mentioned by the authors. The first step requires us to pass the design description and ask the LLM to create a plan. The the generated plan is passed in with the design description again to get the final code. Commented out code was instruction prompting that I tested but did not use when reporting my self planning results. As mentioned in the presentation, self planning did not perform well at all, and I believe that it is because the LLM is overloaded with information and forget the most important part of the entire prompt - the design description.

In [330]:
def augment_text(prompt):
    #Remove last line of prompt 
    prompt = prompt.split("\n")
    prompt = prompt[:-2]

    prompt = "\n".join(prompt)
    out_prompt = "Let's think step by step on how to write the code. Focus on data processing and reason through your answer.\n"
    out_prompt += "Identify edge cases and complex scenarios that you need to consider.\n"
    # out_prompt = "Use the exact module names and signal names as given in the description.\n"
    # out_prompt += "Let's think step by step before writing the code. \n"
    out_prompt += prompt
    
    return out_prompt

augment_text(design_descriptions['accu'])

"Let's think step by step on how to write the code. Focus on data processing and reason through your answer.\nIdentify edge cases and complex scenarios that you need to consider.\nPlease act as a professional verilog designer.\n\nImplement a module to achieve serial input data accumulation output, input is 8bit data. The valid_in will be set to 1 before the first data comes in. Whenever the module receives 4 input data, the data_out outputs 4 received data accumulation results and sets the valid_out to be 1 (will last only 1 cycle).\n\nModule name:  \n    accu               \nInput ports:\n\tclk: Clock input for synchronization.\n\trst_n: Active-low reset signal.\n\tdata_in[7:0]: 8-bit input data for addition.\n\tvalid_in: Input signal indicating readiness for new data.   \nOutput ports:\n    valid_out: Output signal indicating when 4 input data accumulation is reached.\n\tdata_out[9:0]: 10-bit output data representing the accumulated sum.\n\nImplementation:\nWhen valid_in is 1, data_i

Write the output of the first step of self planning for each design into a dictionary

In [ ]:
#Create file based on design name and write the description
for i in tqdm.tqdm(range(0, len(design_name))):
    design = design_name[i]
    send_gpt = design_descriptions[design]
    send_gpt = augment_text(send_gpt)
    completion = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful chatbot"},
                {"role": "user", "content": send_gpt},
            ],
            temperature=0.2
        )
    prompt_gen[design] = completion.choices[0].message.content

